# Datenexploration & Validierung

In [3]:
# Benötigte Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import great_expectations as ge


#  Ausgaben für Plots
plt.rcParams['axes.formatter.use_locale'] = True
plt.style.use('seaborn-v0_8-darkgrid')  

### 4.2 Datensatz laden

In [4]:
# Loan Data Datensatz laden
df = pd.read_csv('../data/raw/loan_data.csv')
print(f"Datensatz erfolgreich geladen. Form: {df.shape}")

Datensatz erfolgreich geladen. Form: (9578, 14)


### 4.3 Erste Dateninspektion


Führen sie eine erste Dateninspektion durch.

In [5]:
df.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


### 4.4 Explorative Datenanalyse

#### 4.4.1 Kategorische Variablen analysieren

Schauen sie sich zunächst die kategorischen Variablen an.
Erstellen sie Balkendiagramme für jede kategorische Variable.

In [6]:
# Identifizierung kategorischer Spalten
kategorische_spalten = df.select_dtypes(include=['object']).columns

In [7]:
print("Kategorische Spalten:")
for spalte in kategorische_spalten:
    print(f"- {spalte}")

Kategorische Spalten:
- purpose


#### 4.4.2 Numerische Variablen analysieren

Analysieren sie nun die numerischen Variablen.
Erstellen Sie Histogramme für die numerischen Variablen und eine Korrelationsmatrix. 

In [8]:
# Identifizierung numerischer Spalten
numerische_spalten = df.select_dtypes(include=['int64', 'float64']).columns

In [9]:
print("numerische_spalten:")
for spalte in numerische_spalten:
    print(f"- {spalte}")

numerische_spalten:
- credit.policy
- int.rate
- installment
- log.annual.inc
- dti
- fico
- days.with.cr.line
- revol.bal
- revol.util
- inq.last.6mths
- delinq.2yrs
- pub.rec
- not.fully.paid


### 4.5 Datenvalidierung mit Great Expectations

#### 4.5.1 Great Expectations Dataset erstellen

In [11]:
context = ge.get_context()

print(context)



{
  "checkpoint_store_name": "checkpoint_store",
  "config_version": 4,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "C:\\Users\\bjoer\\AppData\\Local\\Temp\\tmpikd7lfz1"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {},
  "stores": {
    "expectations_store": {
      "class_name": "ExpectationsStore",
      "store_backend": {
        "class_name": "InMemoryStoreBackend"
      }
    },
    "validation_results_store": {
      "class_name": "ValidationResultsStore",
      "store_backend": {
        "class_name": "InMemoryStoreBackend"
      }
    },
    "checkpoint_store": {
      "class_name": "CheckpointStore",
      "store_backend": {
        "class_name": "InMemoryStoreBackend"
    

In [19]:
# datasource & asset holst du ja schon:
data_source = context.data_sources.get("pandas")
data_asset = data_source.get_asset("pd dataframe asset")

# Batch Definition: erst versuchen zu holen, sonst anlegen
try:
    batch_definition = data_asset.get_batch_definition("batch definition")
except Exception:
    batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")

batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

validator = context.get_validator(
    batch=batch,
    create_expectation_suite_with_name="loan_suite"
)
print(validator)



#### 4.5.2 Grundlegende Erwartungen definieren

Definieren sie Grundlegende Erwartungen an den Datensatz. 

#### 4.5.3 Erwartungssuite erstellen

Erstellen sie eine Erwartungssuite und validieren sie ihre Erwartungen an den Datensatz.

In [28]:
# Spaltenstruktur
validator.expect_table_columns_to_match_ordered_list([
    "credit.policy","purpose","int.rate","installment","log.annual.inc","dti","fico",
    "days.with.cr.line","revol.bal","revol.util","inq.last.6mths","delinq.2yrs","pub.rec","not.fully.paid"
])

# Keine fehlenden Werte in wichtigen Features
for col in ["purpose","int.rate","dti","fico","log.annual.inc","credit.policy","not.fully.paid"]:
    validator.expect_column_values_to_not_be_null(col)

# Binäre Spalten
validator.expect_column_values_to_be_in_set("credit.policy", [0,1])
validator.expect_column_values_to_be_in_set("not.fully.paid", [0,1])

# Kategorien
validator.expect_column_values_to_be_in_set(
    "purpose",
    ["credit_card","debt_consolidation","educational","major_purchase","small_business","all_other"]
)

# Wertebereiche (realistische Finanzwerte)
validator.expect_column_values_to_be_between("fico", min_value=300, max_value=850)
validator.expect_column_values_to_be_between("int.rate", min_value=0, max_value=1)
validator.expect_column_values_to_be_between("installment", min_value=0)
validator.expect_column_values_to_be_between("dti", min_value=0, max_value=100)
validator.expect_column_values_to_be_between("revol.util", min_value=0, max_value=100)

# Einkommen (logarithmiert)
validator.expect_column_values_to_be_between("log.annual.inc", min_value=8, max_value=15)

# Kreditlinien-Dauer
validator.expect_column_values_to_be_between("days.with.cr.line", min_value=0)

# Salden
validator.expect_column_values_to_be_between("revol.bal", min_value=0)

# Zählvariablen (dürfen nicht negativ sein)
for col in ["inq.last.6mths","delinq.2yrs","pub.rec"]:
    validator.expect_column_values_to_be_between(col, min_value=0)

    

for res in result.results:
    if not res.success:
        exp_cfg = res.expectation_config

        exp_type = getattr(exp_cfg, "expectation_type", None) or getattr(exp_cfg, "type", None)
        kwargs = getattr(exp_cfg, "kwargs", {}) or {}

        print(f"- Expectation: {exp_type} on column {kwargs.get('column')}")





c:\Users\bjoer\AppData\Local\Programs\Python\Python311\Lib\site-packages\great_expectations\expectations\expectation.py:1633: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 2/2 [00:00<00:00, 362.81it/s]
c:\Users\bjoer\AppData\Local\Programs\Python\Python311\Lib\site-packages\great_expectations\expectations\expectation.py:1633: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(
Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 659.81it/s] 
c:\Users\bjoer\AppData\Local\Programs\Python\Python311\Lib\site-packages\great_expectations\expectations\expectation.py:1633: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add 

- Expectation: expect_column_values_to_be_in_set on column purpose
- Expectation: expect_column_values_to_be_between on column log.annual.inc
- Expectation: expect_column_values_to_be_between on column revol.util
